In [5]:
"""
This file is intended to identify the optimal method for reconstructing a CAA vector using an SAE model.

It is research code, the exploration code been generated by Claude 3.5 and was not proof-read.

1. Should the code not run, try upgrading transformers in your env:
pip install --upgrade transformers

2. The next cell is a system path fix. It may or may not be necessary to run in your environment.
"""



'\nThis file is intended to identify the optimal method for reconstructing a CAA vector using an SAE model.\n\nIt is research code, the exploration code been generated by Claude 3.5 and was not proof-read.\n\n1. Should the code not run, try upgrading transformers in your env:\npip install --upgrade transformers\n\n2. The next cell is a system path fix. It may or may not be necessary to run in your environment.\n'

In [6]:
# TRANSFORMER / ENVIRONMENT FIX
import sys
sys.path.append('/root/CAA/venv/lib/python3.10/site-packages')
print(sys.path)

['/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/root/CAA/venv/lib/python3.10/site-packages', '/root/CAA/venv/lib/python3.10/site-packages']


In [7]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import time
import torch

from dotenv import load_dotenv
from behaviors import ALL_BEHAVIORS, BASE_DIR, COORDINATE, get_vector_path
from gemma_2_wrapper import Gemma2Wrapper
from generate_vectors import generate_save_vectors_for_behavior
import gemma_vector_analysis
from huggingface_hub import hf_hub_download
import numpy as np
import torch.nn as nn

In [8]:
load_dotenv()
HUGGINGFACE_TOKEN = os.getenv("HF_TOKEN")

In [17]:
# EXPERIMENT 1
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
from huggingface_hub import hf_hub_download

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
layer = 14

sae_model_name = "google/gemma-scope-2b-pt-res"
path_to_params = hf_hub_download(
    repo_id=sae_model_name,
    filename=f"layer_{layer}/width_16k/average_l0_84/params.npz",
    force_download=False,
)

params = np.load(path_to_params)
pt_params = {k: torch.from_numpy(v).to(device) for k, v in params.items()}

class JumpReLUSAE(nn.Module):
    def __init__(self, d_model, d_sae):
        super().__init__()
        self.W_enc = nn.Parameter(torch.zeros(d_model, d_sae))
        self.W_dec = nn.Parameter(torch.zeros(d_sae, d_model))
        self.threshold = nn.Parameter(torch.zeros(d_sae))
        self.b_enc = nn.Parameter(torch.zeros(d_sae))
        self.b_dec = nn.Parameter(torch.zeros(d_model))

    def encode(self, input_acts):
        pre_acts = input_acts @ self.W_enc + self.b_enc
        mask = (pre_acts > self.threshold)
        acts = mask * torch.nn.functional.relu(pre_acts)
        return acts

    def decode(self, acts):
        return acts @ self.W_dec + self.b_dec

    def forward(self, acts):
        acts = self.encode(acts)
        recon = self.decode(acts)
        return recon
        
    def process_caa_vector(self, caa_vector, scale_factor=15, use_leaky_relu=True, negative_slope=0.01):
        caa_vector = caa_vector.to(self.W_enc.device)
        caa_vector_scaled = caa_vector * scale_factor
        adjusted_vector = caa_vector_scaled + self.b_dec
        pre_acts = adjusted_vector @ self.W_enc + self.b_enc
        
        if use_leaky_relu:
            encoded = F.leaky_relu(pre_acts, negative_slope=negative_slope)
        else:
            encoded = F.relu(pre_acts)
        
        decoded = encoded @ self.W_dec + self.b_dec
        decoded_adjusted = decoded - self.b_dec
        decoded_normalized = F.normalize(decoded_adjusted, dim=-1)
        return decoded_normalized

def optimize_parameters(sae, caa_vectors, device):
    best_avg_similarity = 0
    best_params = {}
    
    scale_factors = np.logspace(0, 2, 20)
    use_leaky_relu_options = [True, False]
    negative_slopes = [0.01, 0.1, 0.2]
    
    for scale_factor in scale_factors:
        for use_leaky_relu in use_leaky_relu_options:
            for negative_slope in negative_slopes:
                similarities = []
                
                for behavior, caa_vector in caa_vectors.items():
                    caa_vector = caa_vector.to(device)
                    forwarded_caa_vector = sae.process_caa_vector(caa_vector, scale_factor, use_leaky_relu, negative_slope)
                    
                    cosine_sim = F.cosine_similarity(caa_vector.view(1, -1), forwarded_caa_vector.view(1, -1), dim=1)
                    similarities.append(cosine_sim.item())
                
                avg_similarity = np.mean(similarities)
                
                if avg_similarity > best_avg_similarity:
                    best_avg_similarity = avg_similarity
                    best_params = {
                        'scale_factor': scale_factor,
                        'use_leaky_relu': use_leaky_relu,
                        'negative_slope': negative_slope
                    }
                
                print(f"Scale: {scale_factor:.2f}, Leaky ReLU: {use_leaky_relu}, Slope: {negative_slope:.2f}, Avg Similarity: {avg_similarity:.4f}")

    return best_params, best_avg_similarity

sae = JumpReLUSAE(params['W_enc'].shape[0], params['W_enc'].shape[1])
sae.load_state_dict(pt_params)
sae = sae.to(device)
print("Success")

caa_vectors = {}
for behavior in ALL_BEHAVIORS:
    layer = 14
    behavior = behavior
    model_name_path = 'gemma-2-2b'
    vector_path = gemma_vector_analysis.fix_vector_path(get_vector_path(behavior, layer, model_name_path))
    normalized_dir = os.path.join(BASE_DIR, 'normalized_vectors', behavior)
    
    original_path = os.path.join(normalized_dir, f"vec_layer_{layer}_{model_name_path}.pt")
    original_path = gemma_vector_analysis.fix_vector_path(original_path)
    caa_vector = torch.load(original_path)
    caa_vectors[behavior] = caa_vector

    print(f"Analyzing vector for behavior: {behavior}")
    
    caa_vector = caa_vector.to(device)
    
    forwarded_caa_vector = sae.process_caa_vector(caa_vector)
    print(f"Norm of the forwarded vector: {forwarded_caa_vector.norm().item():.4f}")
    
    sae_vector_dir = '/root/CAA/sae_vector'
    if not os.path.exists(sae_vector_dir):
        os.makedirs(sae_vector_dir)
    
    forwarded_vector_path = os.path.join(sae_vector_dir, f"forwarded_{behavior}.pt")
    torch.save(forwarded_caa_vector.cpu(), forwarded_vector_path)
    print(f"Saved forwarded vector for behavior {behavior} at {forwarded_vector_path}")
    
    caa_vector = caa_vector.to(device)
    forwarded_caa_vector = forwarded_caa_vector.to(device)
    
    caa_vector = caa_vector.view(1, -1)
    forwarded_caa_vector = forwarded_caa_vector.view(1, -1)
    
    cosine_sim = F.cosine_similarity(caa_vector, forwarded_caa_vector, dim=1)
    
    print(f"Cosine similarity between original and forwarded vector: {cosine_sim.item():.4f}")

best_params, best_avg_similarity = optimize_parameters(sae, caa_vectors, device)

print(f"\nBest parameters found:")
print(f"Scale factor: {best_params['scale_factor']:.2f}")
print(f"Use Leaky ReLU: {best_params['use_leaky_relu']}")
print(f"Negative slope: {best_params['negative_slope']:.2f}")
print(f"Best average cosine similarity: {best_avg_similarity:.4f}")

for behavior, caa_vector in caa_vectors.items():
    caa_vector = caa_vector.to(device)
    forwarded_caa_vector = sae.process_caa_vector(
        caa_vector, 
        best_params['scale_factor'], 
        best_params['use_leaky_relu'], 
        best_params['negative_slope']
    )
    
    sae_vector_dir = '/root/CAA/sae_vector'
    if not os.path.exists(sae_vector_dir):
        os.makedirs(sae_vector_dir)
    
    forwarded_vector_path = os.path.join(sae_vector_dir, f"forwarded_{behavior}.pt")
    torch.save(forwarded_caa_vector.cpu(), forwarded_vector_path)
    print(f"Saved optimized forwarded vector for behavior {behavior} at {forwarded_vector_path}")

    cosine_sim = F.cosine_similarity(caa_vector.view(1, -1), forwarded_caa_vector.view(1, -1), dim=1)
    print(f"Optimized cosine similarity for {behavior}: {cosine_sim.item():.4f}")

Success
Analyzing vector for behavior: coordinate-other-ais
Norm of the forwarded vector: 1.0000
Saved forwarded vector for behavior coordinate-other-ais at /root/CAA/sae_vector/forwarded_coordinate-other-ais.pt
Cosine similarity between original and forwarded vector: 0.7098
Analyzing vector for behavior: corrigible-neutral-HHH
Norm of the forwarded vector: 1.0000
Saved forwarded vector for behavior corrigible-neutral-HHH at /root/CAA/sae_vector/forwarded_corrigible-neutral-HHH.pt
Cosine similarity between original and forwarded vector: 0.7304
Analyzing vector for behavior: hallucination
Norm of the forwarded vector: 1.0000
Saved forwarded vector for behavior hallucination at /root/CAA/sae_vector/forwarded_hallucination.pt
Cosine similarity between original and forwarded vector: 0.6907
Analyzing vector for behavior: myopic-reward
Norm of the forwarded vector: 1.0000
Saved forwarded vector for behavior myopic-reward at /root/CAA/sae_vector/forwarded_myopic-reward.pt
Cosine similarity be

In [ ]:
#EXPERIMENT 2
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
from huggingface_hub import hf_hub_download

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
layer = 14

sae_model_name = "google/gemma-scope-2b-pt-res"
path_to_params = hf_hub_download(
    repo_id=sae_model_name,
    filename=f"layer_{layer}/width_16k/average_l0_84/params.npz",
    force_download=False,
)

params = np.load(path_to_params)
pt_params = {k: torch.from_numpy(v).to(device) for k, v in params.items()}

class JumpReLUSAE(nn.Module):
    def __init__(self, d_model, d_sae):
        super().__init__()
        self.W_enc = nn.Parameter(torch.zeros(d_model, d_sae))
        self.W_dec = nn.Parameter(torch.zeros(d_sae, d_model))
        self.threshold = nn.Parameter(torch.zeros(d_sae))
        self.b_enc = nn.Parameter(torch.zeros(d_sae))
        self.b_dec = nn.Parameter(torch.zeros(d_model))

    def encode(self, input_acts):
        pre_acts = input_acts @ self.W_enc + self.b_enc
        mask = (pre_acts > self.threshold)
        acts = mask * torch.nn.functional.relu(pre_acts)
        return acts

    def decode(self, acts):
        return acts @ self.W_dec + self.b_dec

    def forward(self, acts):
        acts = self.encode(acts)
        recon = self.decode(acts)
        return recon
        
    def process_caa_vector(self, caa_vector, scale_factor=15, activation='leaky_relu', activation_param=0.01):
        caa_vector = caa_vector.to(self.W_enc.device)
        caa_vector_scaled = caa_vector * scale_factor
        adjusted_vector = caa_vector_scaled + self.b_dec
        pre_acts = adjusted_vector @ self.W_enc + self.b_enc
        
        if activation == 'leaky_relu':
            encoded = F.leaky_relu(pre_acts, negative_slope=activation_param)
        elif activation == 'elu':
            encoded = F.elu(pre_acts, alpha=activation_param)
        elif activation == 'selu':
            encoded = F.selu(pre_acts)
        else:
            encoded = F.relu(pre_acts)
        
        decoded = encoded @ self.W_dec + self.b_dec
        decoded_adjusted = decoded - self.b_dec
        decoded_normalized = F.normalize(decoded_adjusted, dim=-1)
        return decoded_normalized

def optimize_parameters(sae, caa_vectors, device):
    best_avg_similarity = 0
    best_params = {}
    
    scale_factors = np.logspace(0, 3, 30)  # Expanded range and more granular
    activations = ['leaky_relu', 'elu', 'selu', 'relu']
    activation_params = np.logspace(-3, 0, 10)  # For leaky_relu and elu
    
    for scale_factor in scale_factors:
        for activation in activations:
            if activation in ['leaky_relu', 'elu']:
                params_to_test = activation_params
            else:
                params_to_test = [0]  # Dummy value for selu and relu
            
            for activation_param in params_to_test:
                similarities = []
                
                for behavior, caa_vector in caa_vectors.items():
                    caa_vector = caa_vector.to(device)
                    forwarded_caa_vector = sae.process_caa_vector(caa_vector, scale_factor, activation, activation_param)
                    
                    cosine_sim = F.cosine_similarity(caa_vector.view(1, -1), forwarded_caa_vector.view(1, -1), dim=1)
                    similarities.append(cosine_sim.item())
                
                avg_similarity = np.mean(similarities)
                
                if avg_similarity > best_avg_similarity:
                    best_avg_similarity = avg_similarity
                    best_params = {
                        'scale_factor': scale_factor,
                        'activation': activation,
                        'activation_param': activation_param
                    }
                
                print(f"Scale: {scale_factor:.2f}, Activation: {activation}, Param: {activation_param:.4f}, Avg Similarity: {avg_similarity:.4f}")

    return best_params, best_avg_similarity

sae = JumpReLUSAE(params['W_enc'].shape[0], params['W_enc'].shape[1])
sae.load_state_dict(pt_params)
sae = sae.to(device)
print("Success")

caa_vectors = {}
for behavior in ALL_BEHAVIORS:
    layer = 14
    behavior = behavior
    model_name_path = 'gemma-2-2b'
    vector_path = gemma_vector_analysis.fix_vector_path(get_vector_path(behavior, layer, model_name_path))
    normalized_dir = os.path.join(BASE_DIR, 'normalized_vectors', behavior)
    
    original_path = os.path.join(normalized_dir, f"vec_layer_{layer}_{model_name_path}.pt")
    original_path = gemma_vector_analysis.fix_vector_path(original_path)
    caa_vector = torch.load(original_path)
    caa_vectors[behavior] = caa_vector

    print(f"Analyzing vector for behavior: {behavior}")
    
    caa_vector = caa_vector.to(device)
    
    forwarded_caa_vector = sae.process_caa_vector(caa_vector)
    print(f"Norm of the forwarded vector: {forwarded_caa_vector.norm().item():.4f}")
    
    sae_vector_dir = '/root/CAA/sae_vector'
    if not os.path.exists(sae_vector_dir):
        os.makedirs(sae_vector_dir)
    
    forwarded_vector_path = os.path.join(sae_vector_dir, f"forwarded_{behavior}.pt")
    torch.save(forwarded_caa_vector.cpu(), forwarded_vector_path)
    print(f"Saved forwarded vector for behavior {behavior} at {forwarded_vector_path}")
    
    caa_vector = caa_vector.to(device)
    forwarded_caa_vector = forwarded_caa_vector.to(device)
    
    caa_vector = caa_vector.view(1, -1)
    forwarded_caa_vector = forwarded_caa_vector.view(1, -1)
    
    cosine_sim = F.cosine_similarity(caa_vector, forwarded_caa_vector, dim=1)
    
    print(f"Cosine similarity between original and forwarded vector: {cosine_sim.item():.4f}")

best_params, best_avg_similarity = optimize_parameters(sae, caa_vectors, device)

print(f"\nBest parameters found:")
print(f"Scale factor: {best_params['scale_factor']:.2f}")
print(f"Activation: {best_params['activation']}")
print(f"Activation parameter: {best_params['activation_param']:.4f}")
print(f"Best average cosine similarity: {best_avg_similarity:.4f}")

for behavior, caa_vector in caa_vectors.items():
    caa_vector = caa_vector.to(device)
    forwarded_caa_vector = sae.process_caa_vector(
        caa_vector, 
        best_params['scale_factor'], 
        best_params['activation'], 
        best_params['activation_param']
    )
    
    sae_vector_dir = '/root/CAA/sae_vector'
    if not os.path.exists(sae_vector_dir):
        os.makedirs(sae_vector_dir)
    
    forwarded_vector_path = os.path.join(sae_vector_dir, f"forwarded_{behavior}.pt")
    torch.save(forwarded_caa_vector.cpu(), forwarded_vector_path)
    print(f"Saved optimized forwarded vector for behavior {behavior} at {forwarded_vector_path}")

    cosine_sim = F.cosine_similarity(caa_vector.view(1, -1), forwarded_caa_vector.view(1, -1), dim=1)
    print(f"Optimized cosine similarity for {behavior}: {cosine_sim.item():.4f}")

In [18]:
#EXPERIMENT 3
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
from huggingface_hub import hf_hub_download

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
layer = 14

sae_model_name = "google/gemma-scope-2b-pt-res"
path_to_params = hf_hub_download(
    repo_id=sae_model_name,
    filename=f"layer_{layer}/width_16k/average_l0_84/params.npz",
    force_download=False,
)

params = np.load(path_to_params)
pt_params = {k: torch.from_numpy(v).to(device) for k, v in params.items()}

class JumpReLUSAE(nn.Module):
    def __init__(self, d_model, d_sae):
        super().__init__()
        self.W_enc = nn.Parameter(torch.zeros(d_model, d_sae))
        self.W_dec = nn.Parameter(torch.zeros(d_sae, d_model))
        self.threshold = nn.Parameter(torch.zeros(d_sae))
        self.b_enc = nn.Parameter(torch.zeros(d_sae))
        self.b_dec = nn.Parameter(torch.zeros(d_model))

    def encode(self, input_acts):
        pre_acts = input_acts @ self.W_enc + self.b_enc
        mask = (pre_acts > self.threshold)
        acts = mask * torch.nn.functional.relu(pre_acts)
        return acts

    def decode(self, acts):
        return acts @ self.W_dec + self.b_dec

    def forward(self, acts):
        acts = self.encode(acts)
        recon = self.decode(acts)
        return recon
        
    def process_caa_vector(self, caa_vector, scale_factor=1.0):
        caa_vector = caa_vector.to(self.W_enc.device)
        caa_vector_scaled = caa_vector * scale_factor
        adjusted_vector = caa_vector_scaled + self.b_dec
        encoded = self.encode(adjusted_vector)
        decoded = self.decode(encoded)
        decoded_adjusted = decoded - self.b_dec
        return decoded_adjusted

def optimize_parameters(sae, caa_vectors, device):
    best_avg_similarity = 0
    best_params = {}
    
    scale_factors = np.linspace(0.1, 10, 100)  # More reasonable range
    
    for scale_factor in scale_factors:
        similarities = []
        
        for behavior, caa_vector in caa_vectors.items():
            caa_vector = caa_vector.to(device)
            forwarded_caa_vector = sae.process_caa_vector(caa_vector, scale_factor)
            
            cosine_sim = F.cosine_similarity(caa_vector.view(1, -1), forwarded_caa_vector.view(1, -1), dim=1)
            similarities.append(cosine_sim.item())
        
        avg_similarity = np.mean(similarities)
        
        if avg_similarity > best_avg_similarity:
            best_avg_similarity = avg_similarity
            best_params = {'scale_factor': scale_factor}
        
        print(f"Scale: {scale_factor:.2f}, Avg Similarity: {avg_similarity:.4f}")

    return best_params, best_avg_similarity

sae = JumpReLUSAE(params['W_enc'].shape[0], params['W_enc'].shape[1])
sae.load_state_dict(pt_params)
sae = sae.to(device)
print("SAE loaded successfully")

caa_vectors = {}
for behavior in ALL_BEHAVIORS:
    layer = 14
    model_name_path = 'gemma-2-2b'
    vector_path = gemma_vector_analysis.fix_vector_path(get_vector_path(behavior, layer, model_name_path))
    normalized_dir = os.path.join(BASE_DIR, 'normalized_vectors', behavior)
    
    original_path = os.path.join(normalized_dir, f"vec_layer_{layer}_{model_name_path}.pt")
    original_path = gemma_vector_analysis.fix_vector_path(original_path)
    caa_vector = torch.load(original_path)
    caa_vectors[behavior] = caa_vector

    print(f"Loaded CAA vector for behavior: {behavior}")

print("Optimizing parameters...")
best_params, best_avg_similarity = optimize_parameters(sae, caa_vectors, device)

print(f"\nBest parameters found:")
print(f"Scale factor: {best_params['scale_factor']:.4f}")
print(f"Best average cosine similarity: {best_avg_similarity:.4f}")

print("\nProcessing CAA vectors with optimized parameters:")
for behavior, caa_vector in caa_vectors.items():
    caa_vector = caa_vector.to(device)
    forwarded_caa_vector = sae.process_caa_vector(caa_vector, best_params['scale_factor'])
    
    sae_vector_dir = '/root/CAA/sae_vector'
    if not os.path.exists(sae_vector_dir):
        os.makedirs(sae_vector_dir)
    
    forwarded_vector_path = os.path.join(sae_vector_dir, f"forwarded_{behavior}.pt")
    torch.save(forwarded_caa_vector.cpu(), forwarded_vector_path)
    print(f"Saved processed vector for behavior {behavior} at {forwarded_vector_path}")

    cosine_sim = F.cosine_similarity(caa_vector.view(1, -1), forwarded_caa_vector.view(1, -1), dim=1)
    print(f"Cosine similarity for {behavior}: {cosine_sim.item():.4f}")

    # Calculate and print L2 norm of the original and processed vectors
    original_norm = torch.norm(caa_vector).item()
    processed_norm = torch.norm(forwarded_caa_vector).item()
    print(f"L2 norm - Original: {original_norm:.4f}, Processed: {processed_norm:.4f}")

    # Calculate and print mean and standard deviation of the original and processed vectors
    original_mean = torch.mean(caa_vector).item()
    original_std = torch.std(caa_vector).item()
    processed_mean = torch.mean(forwarded_caa_vector).item()
    processed_std = torch.std(forwarded_caa_vector).item()
    print(f"Mean - Original: {original_mean:.4f}, Processed: {processed_mean:.4f}")
    print(f"Std Dev - Original: {original_std:.4f}, Processed: {processed_std:.4f}")
    print()

SAE loaded successfully
Loaded CAA vector for behavior: coordinate-other-ais
Loaded CAA vector for behavior: corrigible-neutral-HHH
Loaded CAA vector for behavior: hallucination
Loaded CAA vector for behavior: myopic-reward
Loaded CAA vector for behavior: survival-instinct
Loaded CAA vector for behavior: sycophancy
Loaded CAA vector for behavior: refusal
Optimizing parameters...
Scale: 0.10, Avg Similarity: 0.0000
Scale: 0.20, Avg Similarity: 0.0772
Scale: 0.30, Avg Similarity: 0.1449
Scale: 0.40, Avg Similarity: 0.2043
Scale: 0.50, Avg Similarity: 0.2999
Scale: 0.60, Avg Similarity: 0.3741
Scale: 0.70, Avg Similarity: 0.4418
Scale: 0.80, Avg Similarity: 0.4794
Scale: 0.90, Avg Similarity: 0.5145
Scale: 1.00, Avg Similarity: 0.5362
Scale: 1.10, Avg Similarity: 0.5538
Scale: 1.20, Avg Similarity: 0.5803
Scale: 1.30, Avg Similarity: 0.5940
Scale: 1.40, Avg Similarity: 0.6041
Scale: 1.50, Avg Similarity: 0.6096
Scale: 1.60, Avg Similarity: 0.6178
Scale: 1.70, Avg Similarity: 0.6267
Scale:

In [16]:
#EXPERIMENT 4
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
from huggingface_hub import hf_hub_download

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
layer = 14

sae_model_name = "google/gemma-scope-2b-pt-res"
path_to_params = hf_hub_download(
    repo_id=sae_model_name,
    filename=f"layer_{layer}/width_16k/average_l0_84/params.npz",
    force_download=False,
)

params = np.load(path_to_params)
pt_params = {k: torch.from_numpy(v).to(device) for k, v in params.items()}

class JumpReLUSAE(nn.Module):
    def __init__(self, d_model, d_sae):
        super().__init__()
        self.W_enc = nn.Parameter(torch.zeros(d_model, d_sae))
        self.W_dec = nn.Parameter(torch.zeros(d_sae, d_model))
        self.threshold = nn.Parameter(torch.zeros(d_sae))
        self.b_enc = nn.Parameter(torch.zeros(d_sae))
        self.b_dec = nn.Parameter(torch.zeros(d_model))

    def encode(self, input_acts):
        pre_acts = input_acts @ self.W_enc + self.b_enc
        mask = (pre_acts > self.threshold)
        acts = mask * torch.nn.functional.relu(pre_acts)
        return acts

    def decode(self, acts):
        return acts @ self.W_dec + self.b_dec

    def forward(self, acts):
        acts = self.encode(acts)
        recon = self.decode(acts)
        return recon
        
    def process_caa_vector(self, caa_vector, scale_factor=1.0, target_norm=None):
        caa_vector = caa_vector.to(self.W_enc.device)
        
        # Add back the "canceled out" information
        adjusted_vector = caa_vector + self.b_dec
        
        # Scale to match typical residual activation magnitude
        if target_norm is not None:
            current_norm = torch.norm(adjusted_vector)
            adjusted_vector = adjusted_vector * (target_norm / current_norm)
        else:
            adjusted_vector = adjusted_vector * scale_factor
        
        # Process through SAE
        encoded = self.encode(adjusted_vector)
        decoded = self.decode(encoded)
        
        # Normalize the output
        decoded_normalized = F.normalize(decoded, dim=-1)
        
        return decoded_normalized

def optimize_parameters(sae, caa_vectors, device):
    best_avg_similarity = 0
    best_params = {}
    
    scale_factors = np.linspace(0.1, 10, 100)
    target_norms = np.linspace(10, 1000, 100)
    
    for scale_factor in scale_factors:
        for target_norm in target_norms:
            similarities = []
            
            for behavior, caa_vector in caa_vectors.items():
                caa_vector = caa_vector.to(device)
                forwarded_caa_vector = sae.process_caa_vector(caa_vector, scale_factor, target_norm)
                
                cosine_sim = F.cosine_similarity(caa_vector.view(1, -1), forwarded_caa_vector.view(1, -1), dim=1)
                similarities.append(cosine_sim.item())
            
            avg_similarity = np.mean(similarities)
            
            if avg_similarity > best_avg_similarity:
                best_avg_similarity = avg_similarity
                best_params = {'scale_factor': scale_factor, 'target_norm': target_norm}
            
            print(f"Scale: {scale_factor:.2f}, Target Norm: {target_norm:.2f}, Avg Similarity: {avg_similarity:.4f}")

    return best_params, best_avg_similarity

sae = JumpReLUSAE(params['W_enc'].shape[0], params['W_enc'].shape[1])
sae.load_state_dict(pt_params)
sae = sae.to(device)
print("SAE loaded successfully")

caa_vectors = {}
for behavior in ALL_BEHAVIORS:
    layer = 14
    model_name_path = 'gemma-2-2b'
    vector_path = gemma_vector_analysis.fix_vector_path(get_vector_path(behavior, layer, model_name_path))
    normalized_dir = os.path.join(BASE_DIR, 'normalized_vectors', behavior)
    
    original_path = os.path.join(normalized_dir, f"vec_layer_{layer}_{model_name_path}.pt")
    original_path = gemma_vector_analysis.fix_vector_path(original_path)
    caa_vector = torch.load(original_path)
    caa_vectors[behavior] = caa_vector

    print(f"Loaded CAA vector for behavior: {behavior}")

print("Optimizing parameters...")
best_params, best_avg_similarity = optimize_parameters(sae, caa_vectors, device)

#print(f"\nBest parameters found:")
#print(f"Scale factor: {best_params['scale_factor']:.4f}")
#print(f"Target norm: {best_params['target_norm']:.4f}")
#print(f"Best average cosine similarity: {best_avg_similarity:.4f}")

print("\nProcessing CAA vectors with optimized parameters:")
for behavior, caa_vector in caa_vectors.items():
    caa_vector = caa_vector.to(device)
    forwarded_caa_vector = sae.process_caa_vector(caa_vector, best_params['scale_factor'], best_params['target_norm'])
    
    sae_vector_dir = '/root/CAA/sae_vector'
    if not os.path.exists(sae_vector_dir):
        os.makedirs(sae_vector_dir)
    
    forwarded_vector_path = os.path.join(sae_vector_dir, f"forwarded_{behavior}.pt")
    torch.save(forwarded_caa_vector.cpu(), forwarded_vector_path)
    print(f"Saved processed vector for behavior {behavior} at {forwarded_vector_path}")

    cosine_sim = F.cosine_similarity(caa_vector.view(1, -1), forwarded_caa_vector.view(1, -1), dim=1)
    print(f"Cosine similarity for {behavior}: {cosine_sim.item():.4f}")

    # Calculate and print L2 norm of the original and processed vectors
    original_norm = torch.norm(caa_vector).item()
    processed_norm = torch.norm(forwarded_caa_vector).item()
    print(f"L2 norm - Original: {original_norm:.4f}, Processed: {processed_norm:.4f}")

    # Calculate and print mean and standard deviation of the original and processed vectors
    original_mean = torch.mean(caa_vector).item()
    original_std = torch.std(caa_vector).item()
    processed_mean = torch.mean(forwarded_caa_vector).item()
    processed_std = torch.std(forwarded_caa_vector).item()
    print(f"Mean - Original: {original_mean:.4f}, Processed: {processed_mean:.4f}")
    print(f"Std Dev - Original: {original_std:.4f}, Processed: {processed_std:.4f}")
    print()

SAE loaded successfully
Loaded CAA vector for behavior: coordinate-other-ais
Loaded CAA vector for behavior: corrigible-neutral-HHH
Loaded CAA vector for behavior: hallucination
Loaded CAA vector for behavior: myopic-reward
Loaded CAA vector for behavior: survival-instinct
Loaded CAA vector for behavior: sycophancy
Loaded CAA vector for behavior: refusal
Optimizing parameters...
Scale: 0.10, Target Norm: 10.00, Avg Similarity: -0.0067
Scale: 0.10, Target Norm: 20.00, Avg Similarity: -0.0100
Scale: 0.10, Target Norm: 30.00, Avg Similarity: -0.0072
Scale: 0.10, Target Norm: 40.00, Avg Similarity: -0.0068
Scale: 0.10, Target Norm: 50.00, Avg Similarity: -0.0007
Scale: 0.10, Target Norm: 60.00, Avg Similarity: 0.0018
Scale: 0.10, Target Norm: 70.00, Avg Similarity: 0.0087
Scale: 0.10, Target Norm: 80.00, Avg Similarity: 0.0235
Scale: 0.10, Target Norm: 90.00, Avg Similarity: 0.0342
Scale: 0.10, Target Norm: 100.00, Avg Similarity: 0.0501
Scale: 0.10, Target Norm: 110.00, Avg Similarity: 0.

In [13]:
#EXPERIMENT 5
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
from huggingface_hub import hf_hub_download

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
layer = 14

sae_model_name = "google/gemma-scope-2b-pt-res"
path_to_params = hf_hub_download(
    repo_id=sae_model_name,
    filename=f"layer_{layer}/width_16k/average_l0_84/params.npz",
    force_download=False,
)

params = np.load(path_to_params)
pt_params = {k: torch.from_numpy(v).to(device) for k, v in params.items()}

class JumpReLUSAE(nn.Module):
    def __init__(self, d_model, d_sae):
        super().__init__()
        self.W_enc = nn.Parameter(torch.zeros(d_model, d_sae))
        self.W_dec = nn.Parameter(torch.zeros(d_sae, d_model))
        self.threshold = nn.Parameter(torch.zeros(d_sae))
        self.b_enc = nn.Parameter(torch.zeros(d_sae))
        self.b_dec = nn.Parameter(torch.zeros(d_model))

    def encode(self, input_acts):
        pre_acts = input_acts @ self.W_enc + self.b_enc
        mask = (pre_acts > self.threshold)
        acts = mask * torch.nn.functional.relu(pre_acts)
        return acts

    def decode(self, acts):
        return acts @ self.W_dec + self.b_dec

    def forward(self, acts):
        acts = self.encode(acts)
        recon = self.decode(acts)
        return recon
        
    def process_caa_vector(self, caa_vector, scale_factor=1.0, add_bias=False, normalize_input=False):
        caa_vector = caa_vector.to(self.W_enc.device)
        
        if add_bias:
            adjusted_vector = caa_vector + self.b_dec
        else:
            adjusted_vector = caa_vector

        adjusted_vector = adjusted_vector * scale_factor
        
        if normalize_input:
            adjusted_vector = F.normalize(adjusted_vector, dim=-1)
        
        encoded = self.encode(adjusted_vector)
        decoded = self.decode(encoded)
        
        return decoded

def optimize_parameters(sae, caa_vectors, device):
    best_avg_similarity = 0
    best_params = {}
    
    scale_factors = np.logspace(0, 3, 20)
    add_bias_options = [True, False]
    normalize_input_options = [True, False]
    
    for scale_factor in scale_factors:
        for add_bias in add_bias_options:
            for normalize_input in normalize_input_options:
                similarities = []
                
                for behavior, caa_vector in caa_vectors.items():
                    caa_vector = caa_vector.to(device)
                    forwarded_caa_vector = sae.process_caa_vector(caa_vector, scale_factor, add_bias, normalize_input)
                    
                    cosine_sim = F.cosine_similarity(caa_vector.view(1, -1), forwarded_caa_vector.view(1, -1), dim=1)
                    similarities.append(cosine_sim.item())
                
                avg_similarity = np.mean(similarities)
                
                if avg_similarity > best_avg_similarity:
                    best_avg_similarity = avg_similarity
                    best_params = {
                        'scale_factor': scale_factor,
                        'add_bias': add_bias,
                        'normalize_input': normalize_input
                    }
                
                print(f"Scale: {scale_factor:.2f}, Add Bias: {add_bias}, Normalize Input: {normalize_input}, Avg Similarity: {avg_similarity:.4f}")

    return best_params, best_avg_similarity

sae = JumpReLUSAE(params['W_enc'].shape[0], params['W_enc'].shape[1])
sae.load_state_dict(pt_params)
sae = sae.to(device)
print("SAE loaded successfully")

print(f"SAE model width: Encoder width = {sae.W_enc.shape[1]}, Decoder width = {sae.W_dec.shape[0]}")
print("Decoder bias (b_dec) statistics:")
print(f"Shape: {sae.b_dec.shape}")
print(f"Mean: {sae.b_dec.mean().item():.4f}")
print(f"Std Dev: {sae.b_dec.std().item():.4f}")
print(f"Min: {sae.b_dec.min().item():.4f}")
print(f"Max: {sae.b_dec.max().item():.4f}")

caa_vectors = {}
for behavior in ALL_BEHAVIORS:
    layer = 14
    model_name_path = 'gemma-2-2b'
    vector_path = gemma_vector_analysis.fix_vector_path(get_vector_path(behavior, layer, model_name_path))
    normalized_dir = os.path.join(BASE_DIR, 'normalized_vectors', behavior)
    
    original_path = os.path.join(normalized_dir, f"vec_layer_{layer}_{model_name_path}.pt")
    original_path = gemma_vector_analysis.fix_vector_path(original_path)
    caa_vector = torch.load(original_path)
    caa_vectors[behavior] = caa_vector

    print(f"Loaded CAA vector for behavior: {behavior}")

print("Optimizing parameters...")
best_params, best_avg_similarity = optimize_parameters(sae, caa_vectors, device)

print(f"\nBest parameters found:")
print(f"Scale factor: {best_params['scale_factor']:.4f}")
print(f"Add bias: {best_params['add_bias']}")
print(f"Normalize input: {best_params['normalize_input']}")
print(f"Best average cosine similarity: {best_avg_similarity:.4f}")

print("\nProcessing CAA vectors with optimized parameters:")
for behavior, caa_vector in caa_vectors.items():
    caa_vector = caa_vector.to(device)
    forwarded_caa_vector = sae.process_caa_vector(
        caa_vector,
        best_params['scale_factor'],
        best_params['add_bias'],
        best_params['normalize_input']
    )
    
    sae_vector_dir = '/root/CAA/sae_vector'
    if not os.path.exists(sae_vector_dir):
        os.makedirs(sae_vector_dir)
    
    forwarded_vector_path = os.path.join(sae_vector_dir, f"forwarded_{behavior}.pt")
    torch.save(forwarded_caa_vector.cpu(), forwarded_vector_path)
    print(f"Saved processed vector for behavior {behavior} at {forwarded_vector_path}")

    cosine_sim = F.cosine_similarity(caa_vector.view(1, -1), forwarded_caa_vector.view(1, -1), dim=1)
    print(f"Cosine similarity for {behavior}: {cosine_sim.item():.4f}")

    original_norm = torch.norm(caa_vector).item()
    processed_norm = torch.norm(forwarded_caa_vector).item()
    print(f"L2 norm - Original: {original_norm:.4f}, Processed: {processed_norm:.4f}")

    original_mean = torch.mean(caa_vector).item()
    original_std = torch.std(caa_vector).item()
    processed_mean = torch.mean(forwarded_caa_vector).item()
    processed_std = torch.std(forwarded_caa_vector).item()
    print(f"Mean - Original: {original_mean:.4f}, Processed: {processed_mean:.4f}")
    print(f"Std Dev - Original: {original_std:.4f}, Processed: {processed_std:.4f}")
    print()

SAE loaded successfully
SAE model width: Encoder width = 16384, Decoder width = 16384
Decoder bias (b_dec) statistics:
Shape: torch.Size([2304])
Mean: 0.0473
Std Dev: 2.2681
Min: -16.8032
Max: 93.9256
Loaded CAA vector for behavior: coordinate-other-ais
Loaded CAA vector for behavior: corrigible-neutral-HHH
Loaded CAA vector for behavior: hallucination
Loaded CAA vector for behavior: myopic-reward
Loaded CAA vector for behavior: survival-instinct
Loaded CAA vector for behavior: sycophancy
Loaded CAA vector for behavior: refusal
Optimizing parameters...
Scale: 1.00, Add Bias: True, Normalize Input: True, Avg Similarity: -0.0019
Scale: 1.00, Add Bias: True, Normalize Input: False, Avg Similarity: 0.0681
Scale: 1.00, Add Bias: False, Normalize Input: True, Avg Similarity: -0.0013
Scale: 1.00, Add Bias: False, Normalize Input: False, Avg Similarity: 0.2687
Scale: 1.44, Add Bias: True, Normalize Input: True, Avg Similarity: -0.0019
Scale: 1.44, Add Bias: True, Normalize Input: False, Avg Si

In [14]:
#EXPERIMENT 6 (simple model only using scale)
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
from huggingface_hub import hf_hub_download

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
layer = 14

sae_model_name = "google/gemma-scope-2b-pt-res"
path_to_params = hf_hub_download(
    repo_id=sae_model_name,
    filename=f"layer_{layer}/width_16k/average_l0_84/params.npz",
    force_download=False,
)

params = np.load(path_to_params)
pt_params = {k: torch.from_numpy(v).to(device) for k, v in params.items()}

class JumpReLUSAE(nn.Module):
    def __init__(self, d_model, d_sae):
        super().__init__()
        self.W_enc = nn.Parameter(torch.zeros(d_model, d_sae))
        self.W_dec = nn.Parameter(torch.zeros(d_sae, d_model))
        self.threshold = nn.Parameter(torch.zeros(d_sae))
        self.b_enc = nn.Parameter(torch.zeros(d_sae))
        self.b_dec = nn.Parameter(torch.zeros(d_model))

    def encode(self, input_acts):
        pre_acts = input_acts @ self.W_enc + self.b_enc
        mask = (pre_acts > self.threshold)
        acts = mask * torch.nn.functional.relu(pre_acts)
        return acts

    def decode(self, acts):
        return acts @ self.W_dec + self.b_dec

    def forward(self, acts):
        acts = self.encode(acts)
        recon = self.decode(acts)
        return recon
        
    def process_caa_vector(self, caa_vector, scale_factor=4.2813):
        caa_vector = caa_vector.to(self.W_enc.device)
        adjusted_vector = caa_vector * scale_factor
        encoded = self.encode(adjusted_vector)
        decoded = self.decode(encoded)
        return decoded

def process_caa_vectors(sae, caa_vectors, scale_factor):
    processed_vectors = {}
    similarities = []
    
    for behavior, caa_vector in caa_vectors.items():
        caa_vector = caa_vector.to(device)
        processed_vector = sae.process_caa_vector(caa_vector, scale_factor)
        
        cosine_sim = F.cosine_similarity(caa_vector.view(1, -1), processed_vector.view(1, -1), dim=1)
        similarities.append(cosine_sim.item())
        
        processed_vectors[behavior] = processed_vector.cpu()
        
        print(f"Processed vector for behavior: {behavior}")
        print(f"Cosine similarity: {cosine_sim.item():.4f}")
        print(f"Original L2 norm: {torch.norm(caa_vector).item():.4f}")
        print(f"Processed L2 norm: {torch.norm(processed_vector).item():.4f}")
        print(f"Original mean: {torch.mean(caa_vector).item():.4f}")
        print(f"Processed mean: {torch.mean(processed_vector).item():.4f}")
        print(f"Original std dev: {torch.std(caa_vector).item():.4f}")
        print(f"Processed std dev: {torch.std(processed_vector).item():.4f}")
        print()
    
    avg_similarity = np.mean(similarities)
    print(f"Average cosine similarity: {avg_similarity:.4f}")
    
    return processed_vectors

# Load the SAE model
sae = JumpReLUSAE(params['W_enc'].shape[0], params['W_enc'].shape[1])
sae.load_state_dict(pt_params)
sae = sae.to(device)
print("SAE loaded successfully")

# Load CAA vectors
caa_vectors = {}
for behavior in ALL_BEHAVIORS:
    layer = 14
    model_name_path = 'gemma-2-2b'
    vector_path = gemma_vector_analysis.fix_vector_path(get_vector_path(behavior, layer, model_name_path))
    normalized_dir = os.path.join(BASE_DIR, 'normalized_vectors', behavior)
    
    original_path = os.path.join(normalized_dir, f"vec_layer_{layer}_{model_name_path}.pt")
    original_path = gemma_vector_analysis.fix_vector_path(original_path)
    caa_vector = torch.load(original_path)
    caa_vectors[behavior] = caa_vector
    print(f"Loaded CAA vector for behavior: {behavior}")

# Process CAA vectors
scale_factor = 4.2813  # This value was found to work well in previous experiments
processed_vectors = process_caa_vectors(sae, caa_vectors, scale_factor)

# Save processed vectors
sae_vector_dir = '/root/CAA/sae_vector'
if not os.path.exists(sae_vector_dir):
    os.makedirs(sae_vector_dir)

for behavior, processed_vector in processed_vectors.items():
    save_path = os.path.join(sae_vector_dir, f"processed_{behavior}.pt")
    torch.save(processed_vector, save_path)
    print(f"Saved processed vector for behavior {behavior} at {save_path}")

print("CAA vector processing complete.")

SAE loaded successfully
Loaded CAA vector for behavior: coordinate-other-ais
Loaded CAA vector for behavior: corrigible-neutral-HHH
Loaded CAA vector for behavior: hallucination
Loaded CAA vector for behavior: myopic-reward
Loaded CAA vector for behavior: survival-instinct
Loaded CAA vector for behavior: sycophancy
Loaded CAA vector for behavior: refusal
Processed vector for behavior: coordinate-other-ais
Cosine similarity: 0.7476
Original L2 norm: 44.3626
Processed L2 norm: 354.6800
Original mean: 0.0521
Processed mean: 0.3476
Original std dev: 0.9230
Processed std dev: 7.3826

Processed vector for behavior: corrigible-neutral-HHH
Cosine similarity: 0.7543
Original L2 norm: 44.3626
Processed L2 norm: 344.2238
Original mean: 0.0128
Processed mean: 0.1952
Original std dev: 0.9243
Processed std dev: 7.1702

Processed vector for behavior: hallucination
Cosine similarity: 0.7357
Original L2 norm: 44.3626
Processed L2 norm: 379.2407
Original mean: 0.0288
Processed mean: 0.4702
Original std 

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np
from huggingface_hub import hf_hub_download

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
layer = 14

sae_model_name = "google/gemma-scope-2b-pt-res"
path_to_params = hf_hub_download(
    repo_id=sae_model_name,
    filename=f"layer_{layer}/width_16k/average_l0_84/params.npz",
    force_download=False,
)

params = np.load(path_to_params)
pt_params = {k: torch.from_numpy(v).to(device) for k, v in params.items()}

class JumpReLUSAE(nn.Module):
    def __init__(self, d_model, d_sae):
        super().__init__()
        self.W_enc = nn.Parameter(torch.zeros(d_model, d_sae))
        self.W_dec = nn.Parameter(torch.zeros(d_sae, d_model))
        self.threshold = nn.Parameter(torch.zeros(d_sae))
        self.b_enc = nn.Parameter(torch.zeros(d_sae))
        self.b_dec = nn.Parameter(torch.zeros(d_model))

    def encode(self, input_acts):
        pre_acts = input_acts @ self.W_enc + self.b_enc
        mask = (pre_acts > self.threshold)
        acts = mask * torch.nn.functional.relu(pre_acts)
        return acts

    def decode(self, acts):
        return acts @ self.W_dec + self.b_dec

    def forward(self, acts):
        acts = self.encode(acts)
        recon = self.decode(acts)
        return recon
        
    def process_caa_vector(self, caa_vector):
        caa_vector = caa_vector.to(self.W_enc.device)
        decoder_bias = self.b_dec
        
        # Compute SAE(caa_vector + decoder_bias) and SAE(-caa_vector + decoder_bias)
        positive_encoded = self.encode(caa_vector + decoder_bias)
        negative_encoded = self.encode(-caa_vector + decoder_bias)
        
        positive_decoded = self.decode(positive_encoded)
        negative_decoded = self.decode(negative_encoded)
        
        # Calculate (positive - negative) / 2
        result_vector = (positive_decoded - negative_decoded) / 2
        
        # Calculate cosine similarity between result_vector and caa_vector
        cosine_sim = F.cosine_similarity(result_vector.view(1, -1), caa_vector.view(1, -1), dim=1).item()
        
        return result_vector, cosine_sim

def process_caa_vectors(sae, caa_vectors):
    processed_vectors = {}
    similarities = []
    
    for behavior, caa_vector in caa_vectors.items():
        caa_vector = caa_vector.to(device)
        
        # Unpack the tuple returned by sae.process_caa_vector
        processed_vector, cosine_sim = sae.process_caa_vector(caa_vector)
        
        similarities.append(cosine_sim)
        
        processed_vectors[behavior] = processed_vector.cpu()
        
        print(f"Processed vector for behavior: {behavior}")
        print(f"Cosine similarity: {cosine_sim:.4f}")
        print(f"Original L2 norm: {torch.norm(caa_vector).item():.4f}")
        print(f"Processed L2 norm: {torch.norm(processed_vector).item():.4f}")
        print(f"Original mean: {torch.mean(caa_vector).item():.4f}")
        print(f"Processed mean: {torch.mean(processed_vector).item():.4f}")
        print(f"Original std dev: {torch.std(caa_vector).item():.4f}")
        print(f"Processed std dev: {torch.std(processed_vector).item():.4f}")
        print()
    
    avg_similarity = np.mean(similarities)
    print(f"Average cosine similarity: {avg_similarity:.4f}")
    
    return processed_vectors

# Load the SAE model
sae = JumpReLUSAE(params['W_enc'].shape[0], params['W_enc'].shape[1])
sae.load_state_dict(pt_params)
sae = sae.to(device)
print("SAE loaded successfully")

# Load CAA vectors
caa_vectors = {}
for behavior in ALL_BEHAVIORS:
    layer = 14
    model_name_path = 'gemma-2-2b'
    vector_path = gemma_vector_analysis.fix_vector_path(get_vector_path(behavior, layer, model_name_path))
    normalized_dir = os.path.join(BASE_DIR, 'normalized_vectors', behavior)
    
    original_path = os.path.join(normalized_dir, f"vec_layer_{layer}_{model_name_path}.pt")
    original_path = gemma_vector_analysis.fix_vector_path(original_path)
    caa_vector = torch.load(original_path)
    caa_vectors[behavior] = caa_vector
    print(f"Loaded CAA vector for behavior: {behavior}")

# Process CAA vectors
processed_vectors = process_caa_vectors(sae, caa_vectors)

# Save processed vectors
sae_vector_dir = '/root/CAA/sae_vector'
if not os.path.exists(sae_vector_dir):
    os.makedirs(sae_vector_dir)

for behavior, processed_vector in processed_vectors.items():
    save_path = os.path.join(sae_vector_dir, f"processed_{behavior}.pt")
    torch.save(processed_vector, save_path)
    print(f"Saved processed vector for behavior {behavior} at {save_path}")

print("CAA vector processing complete.")


SAE loaded successfully
Loaded CAA vector for behavior: coordinate-other-ais
Loaded CAA vector for behavior: corrigible-neutral-HHH
Loaded CAA vector for behavior: hallucination
Loaded CAA vector for behavior: myopic-reward
Loaded CAA vector for behavior: survival-instinct
Loaded CAA vector for behavior: sycophancy
Loaded CAA vector for behavior: refusal
Processed vector for behavior: coordinate-other-ais
Cosine similarity: 0.7200
Original L2 norm: 44.3626
Processed L2 norm: 21.7290
Original mean: 0.0521
Processed mean: 0.0035
Original std dev: 0.9230
Processed std dev: 0.4528

Processed vector for behavior: corrigible-neutral-HHH
Cosine similarity: 0.7304
Original L2 norm: 44.3626
Processed L2 norm: 24.5254
Original mean: 0.0128
Processed mean: 0.0032
Original std dev: 0.9243
Processed std dev: 0.5110

Processed vector for behavior: hallucination
Cosine similarity: 0.5983
Original L2 norm: 44.3626
Processed L2 norm: 15.9928
Original mean: 0.0288
Processed mean: -0.0008
Original std de

In [15]:
#VERIFY SAE SELECTED
print(f"SAE model width: Encoder width = {sae.W_enc.shape[1]}, Decoder width = {sae.W_dec.shape[0]}")
print(sae.b_dec)

SAE model width: Encoder width = 16384, Decoder width = 16384
Parameter containing:
tensor([-0.1614, -0.1302, -0.0954,  ...,  0.6045, -0.3325, -0.1574],
       device='cuda:0', requires_grad=True)
